In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
157,AFG,Asia,Afghanistan,2020-06-15,24766.0,664.0,471.0,20.0,636.195,17.057,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50
256,ALB,Europe,Albania,2020-06-15,1521.0,57.0,36.0,0.0,528.529,19.807,...,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89
419,DZA,Africa,Algeria,2020-06-15,10919.0,109.0,767.0,7.0,249.002,2.486,...,6.211,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-14,France,157499.0,0.0,29429.0,0.0,0.0,0.0
0,2020-06-14,China,84365.0,0.0,4650.0,0.0,0.0,0.0
0,2020-06-14,Italy,236954.0,0.0,34378.0,0.0,0.0,0.0
0,2020-06-14,Spain,244145.0,0.0,27146.0,0.0,0.0,0.0
0,2020-06-14,United States,2101918.0,0.0,116420.0,0.0,0.0,0.0
0,2020-06-14,World,7919643.0,0.0,435338.0,0.0,0.0,0.0
0,2020-06-14,United Kingdom,296261.0,0.0,41909.0,0.0,0.0,0.0
0,2020-06-14,Germany,186695.0,0.0,8815.0,0.0,0.0,0.0
0,2020-06-14,Iran,187569.0,0.0,8809.0,0.0,0.0,0.0
0,2020-06-14,Turkey,177799.0,0.0,4806.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-15,France,157499.0,157220.0,29429.0,29407.0,279.0,22.0
0,2020-06-15,China,84365.0,84335.0,4650.0,4638.0,30.0,12.0
0,2020-06-15,Italy,236954.0,236989.0,34378.0,34345.0,-35.0,33.0
0,2020-06-14,Spain,244145.0,243928.0,27146.0,27136.0,217.0,10.0
0,2020-06-15,United States,2101918.0,2094069.0,116420.0,115732.0,7849.0,688.0
0,2020-06-15,World,7919643.0,7882230.0,435338.0,433259.0,37413.0,2079.0
0,2020-06-15,United Kingdom,296261.0,295889.0,41909.0,41698.0,372.0,211.0
0,2020-06-15,Germany,186695.0,186461.0,8815.0,8791.0,234.0,24.0
0,2020-06-15,Iran,187569.0,187427.0,8809.0,8837.0,142.0,-28.0
0,2020-06-15,Turkey,177799.0,178239.0,4806.0,4807.0,-440.0,-1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")